In [32]:
from langchain.document_loaders import PyPDFLoader

# 파일 경로 설정 (백슬래시 문제 해결을 위해 r"" 사용)
pdf_path = r"금융\2025년 중소벤처기업부 소관 소상공인 정책자금 융자계획 공고.pdf"

# 지원사업명 저장 (지원경로를 분할하여 사용)
title = pdf_path.split('\\')[1].split('.')[0]

# PyPDFLoader를 사용하여 PDF 파일 로드
loader = PyPDFLoader(pdf_path)
documents = loader.load()

# 전체 텍스트를 하나의 문자열로 저장
full_text = "\n".join([doc.page_content for doc in documents])

# 추출한 텍스트 출력
print(full_text)

- 1 -
중소벤처기업부공고제2024–622호｢
소상공인보호및지원에관한법률｣
제9조에의한“2025년도중소벤처기업부소관소상공인정책자금융자계획”을다음과같이공고합니다. 2024년12월26일중소벤처기업부장관소상공인시장진흥공단이사장
2025년중소벤처기업부소관소상공인정책자금융자계획Ⅰ 공통사항□융자대상◦｢
소상공인기본법｣
제2조및같은법시행령제3조의소상공인    * 세부사항은 각 자금에서 규정하고, 주된 사업의 업종이 융자제외 대상 업종(별표1)에 해당하는 경우에는 융자대상에서 제외□융자한도및금리◦(융자한도)소진공정책자금대출잔액과신규대출예정액을합산하여기업당운전자금5억원이내에서운용(시설자금포함시10억원이내)    * 긴급경영안정자금은 융자한도와 별도로 운용
- 2 -
◦(대출금리)정책자금기준금리(분기별변동금리)에사업별가산금리를적용하며,일부자금의경우고정금리를적용 -기존대출기업도정책자금기준금리변동에따라대출금리가변동되며(일부자금제외),대출금리는정부정책에따라변경가능 -정책자금기준금리및분기별대출금리는소상공인시장진흥공단(이하‘소진공’이라한다)홈페이지(www.semas.or.kr)에공지• 사업별 대출금리 등 세부사항은 각 자금에서 규정 -유형별0.1%p의금리감면을지원하는금리우대제도를운영하고,최대0.4%p까지금리우대지원(‘25년정책자금신청소상공인)< 소상공인정책자금 우대금리 적용대상 >
유형 구분정책 우대-소진공 또는 은행권 컨설팅을 받은 업체, 제로페이 가맹점, 디지털 온누리상품권(카드형‧모바일형) 가맹점정책 배려-여성기업, 장애인기업, 일회용품 사용규제 적응 우수기업사회안전망-자영업자 고용보험, 전통시장 화재공제, 풍수해보험, 노란우산공제성실상환-소진공 직접대출 성실상환 소상공인     * 동일 유형 내 우대금리 중복 적용 불가    ** 고정금리(장애인기업지원자금, 긴급경영안정자금 내 재해자금, 대환대출 등) 상품 제외□융자방식◦(직접대출)소진공이융자접수→평가→대출실행까지진행◦(대리대출)소진공에서정책자금지원대상확인후,금융기관에서보증서부,신용또는담보부대출실행     * 보증서부 대출은 

In [33]:
keyword_dictionary = {
                        '신청기간' : ['신청 기간', '접수 기간'],
                        '신청방법' : ['지원 대상', '사업 대상', '융자 대상'],
                        '지원제외' : ['지원 제외','지원 취소', '제외 대상', '제한 대상'],
                        '유의사항' : ['의무사항', '지원 취소 및 제외', '기타 안내사항'],
                        '제출 서류' : ['필요 서류', '서류 제출 목록'],
                        '선정 절차' : ['선정 절차', '평가 방법', '평가 방향', '우대 사항', '평가 방향', '선정 기준'],
                        '지원 내용' : ['지원 내용', '규모', '분야 및 대상', '범위 및 형태', '세부지원내용'],
                        '문의처' : ['문의 전화', '연락처', '지원 문의처', '문의 및 상담', '접수 및 문의'],
                        '기타' : ['기타']
                        }

In [34]:
base_template = {f'''
📌 **[{title}] 요약 안내**

📅 **신청기간**  

📝 **신청방법**  

🚫 **지원제외 대상**   

⚠ **유의사항**  

📑 **제출서류**  

📌 **선정절차**  

💰 **지원내용**  

📞 **문의처**  

📢 **기타 안내**  
'''
}

In [12]:
from openai import OpenAI
from getpass import getpass
import json
import re
from tqdm import tqdm

In [14]:
# 개인의 API key를 입력
MY_API_KEY = getpass("OpenAI API Key")

#OpenAI 객체 생성
client = OpenAI(api_key=MY_API_KEY)

In [35]:
content = f'''당신은 파일 요약 전문가입니다. 사용자가 요구하는 파일을 제공하는 템플릿 양식에 맞게 요약해야합니다.
파일에 없는 내용은 공백으로 남겨두십시오.
요구하는 파일은 양식이 일정하지 않습니다. 정의된 사전을 선언해두었으니, 사전에 해당하는 단어가 나오면, 참고하여 요약해주십시오.

제공하는 템플릿 양식 : {base_template}
단어 정의 사전 : {keyword_dictionary}
'''
user_content = f'''{full_text}를 요약해줘'''

completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'system','content':content},
                                                        {'role':'user','content':user_content}],
                                                        temperature=0)

print(completion.choices[0].message.content)

📌 **[2025년 중소벤처기업부 소관 소상공인 정책자금 융자계획 공고] 요약 안내**

📅 **신청기간**  
- 대리대출: 1월 2일 시작  
- 직접대출: 1월 6일 시작  

📝 **신청방법**  
- 온라인: 소상공인정책자금사이트(https://ols.semas.or.kr)  
- 오프라인: 소상공인시장진흥공단 지역센터 방문  

🚫 **지원제외 대상**  
- 세금 체납 중인 소상공인  
- 신용도 판단 정보 등록 소상공인  
- 허위 또는 부정한 방법으로 융자 신청한 소상공인  
- 사회적 물의를 일으킨 소상공인  
- 휴·폐업 중인 소상공인  
- 정책자금 융자 제외 업종 영위 소상공인  
- 최근 5년 이내 정책자금 3회 이상 지원받은 소상공인  

⚠ **유의사항**  
- 대출 후 자금 집행 여부 점검  
- 용도 외 사용 시 자금 조기 회수  

📑 **제출서류**  

📌 **선정절차**  
- 직접대출: 소진공에서 평가 후 대출 실행  
- 대리대출: 소진공 확인 후 금융기관에서 대출 실행  

💰 **지원내용**  
- 융자한도: 운전자금 5억원 이내, 시설자금 포함 시 10억원 이내  
- 대출금리: 정책자금 기준금리 + 가산금리  
- 금리 우대제도 운영  

📞 **문의처**  
- 중소기업통합콜센터: ☎1357  
- 소상공인시장진흥공단 지역센터 연락처  

📢 **기타 안내**  
